In [1]:
!curl https://conceptnet.s3.amazonaws.com/downloads/2017/numberbatch/numberbatch-en-17.06.txt.gz --output numberbatch-en-17.06.txt.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  257M  100  257M    0     0  1955k      0  0:02:14  0:02:14 --:--:-- 2770k6k


https://github.com/loretoparisi/word2vec-twitter

In [5]:
#!gunzip numberbatch-en-17.06.txt.gz

In [376]:
!curl http://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip --output glove.twitter.27B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   326  100   326    0     0    765      0 --:--:-- --:--:-- --:--:--   765


In [377]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [382]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

tmp_file = get_tmpfile("glove_word2vec.txt")
_ = glove2word2vec('glove.twitter.27B.200d.txt', tmp_file)

glove = KeyedVectors.load_word2vec_format(tmp_file)

In [389]:
glove.most_similar(['glove'])

[('gloves', 0.7430825233459473),
 ('helmet', 0.6035714149475098),
 ('cleats', 0.5549577474594116),
 ('shoe', 0.5447742938995361),
 ('boot', 0.5390670299530029),
 ('leather', 0.5314798355102539),
 ('hand', 0.5252668857574463),
 ('pads', 0.523605227470398),
 ('gear', 0.5217989683151245),
 ('coat', 0.5216744542121887)]

In [ ]:
numberbatch = KeyedVectors.load_word2vec_format("numberbatch-en-17.06.txt.gz", binary=False)

In [4]:
numberbatch.most_similar(['functor'])

[('contravariant_functor', 0.9674091935157776),
 ('forgetful_functor', 0.9666687250137329),
 ('yoneda_embedding', 0.9497337937355042),
 ('endofunctor', 0.9360368847846985),
 ('representable_functor', 0.9314213991165161),
 ('cofunctor', 0.9296932220458984),
 ('natural_transformation', 0.9164144992828369),
 ('yoneda_lemma', 0.9022417664527893),
 ('coaugmentation', 0.8871707320213318),
 ('category_theory', 0.8751257658004761)]

In [5]:
from collections import namedtuple

Phrase = namedtuple('Phrase', 'original candidate label')
Token = namedtuple('Token', 'text tags')

def split_tokens(sent):
    tokens = []
    for token in sent.split():
        tags = token.split('/')
        tokens.append(Token(tags[0].lower(), tuple(tags[1:])))
    return tokens


def readData(filename, eval_label, ignoreNone):
    data = []
    with open(filename) as f:
        for line in f:
            fields = line.strip().split('\t')
            if len(fields) == 7:
                (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = fields
            else:
                continue
            label = eval_label(judge)
            if ((label is None) and ignoreNone):
                continue
            data.append(Phrase(split_tokens(origsenttag), split_tokens(candsenttag), label))
    
    return data
                
def eval_amt_label(label):
    nYes = eval(label)[0]            
    
    if nYes >= 3:
        return True
    elif nYes <= 1:
        return False
    
    return None

def eval_expert_label(label):
    nYes = int(label[0])
    
    if nYes >= 4:
        return True
    elif nYes <= 2:
        return False
    
    return None


def readTrainData(filename):
    return readData(filename, eval_amt_label, True)

def readTestData(filename):
    return readData(filename, eval_expert_label, False)

In [6]:
train_data = readTrainData("SemEval-PIT2015-py3/data/train.data")
dev_data = readTrainData("SemEval-PIT2015-py3/data/dev.data")
test_data = [p for p in readTestData("SemEval-PIT2015-py3/data/test.data") if p.label is not None]

In [228]:
len(test_data)

838

In [7]:
train_data[:3]

[Phrase(original=[Token(text='ej', tags=('B-person', 'NNP', 'B-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'B-VP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='1st', tags=('O', 'CD', 'I-NP', 'O')), Token(text='qb', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='to', tags=('O', 'TO', 'B-VP', 'O')), Token(text='go', tags=('O', 'VB', 'I-VP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'I-EVENT')), Token(text='this', tags=('O', 'DT', 'B-NP', 'O')), Token(text='draft', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='but', tags=('O', 'CC', 'O', 'O')), Token(text='my', tags=('O', 'PRP$', 'B-NP', 'O')), Token(text='bro', tags=('O', 'NN', 'I-NP', 'O')), Token(text='from', tags=('O', 'IN', 'B-PP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='757', tags=('O', 'CD', 'I-NP', 'O')), Token(text='ej', tags=('B-person', 'NNP', 'I-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'I-NP', 'O')), Token(text='is', tags=(

In [8]:
test_data[:3]

[Phrase(original=[Token(text='all', tags=('O', 'DT', 'B-NP', 'O')), Token(text='the', tags=('O', 'DT', 'I-NP', 'O')), Token(text='home', tags=('O', 'NN', 'I-NP', 'O')), Token(text='alones', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='watching', tags=('O', 'VBG', 'I-VP', 'B-EVENT')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='last', tags=('O', 'JJ', 'I-NP', 'O')), Token(text='rap', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='battle', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'O')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='nevr', tags=('O', 'NN', 'I-NP', 'O')), Token(text='gets', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='old', tags=('O', 'JJ', 'B-NP', 'O')), Token(text='ahah', tags=('O', 'JJ', 'I-NP', 'O'))], label=False),
 Phrase(original=[To

In [453]:
from nltk.corpus import stopwords
from tqdm import tqdm_notebook

NUM = set(['1st'])

def clean_sent(sent):
    new_sent = []
    for token in sent:
        if token.tags[0].startswith('B-'):
            new_sent.append(token.tags[0].split('-')[1])
            continue
        if token.tags[0].startswith('I-') or token.text in stopwords.words('english'):
            continue
        if token.tags[1] == 'CD' or token.text in NUM or token.text.isnumeric():
            new_sent.append('number')
            continue
        new_sent.append(token.text)
                            
    return new_sent

def clean_data(data):
    return [Phrase(clean_sent(phrase.original), clean_sent(phrase.candidate), phrase.label) \
            for phrase in tqdm_notebook(data)]

In [454]:
clean_train_data = clean_data(train_data)

In [455]:
clean_dev_data = clean_data(dev_data)

In [456]:
clean_test_data = clean_data(test_data)

In [457]:
clean_train_data[:3]

[Phrase(original=['person', 'number', 'qb', 'go', 'draft'], candidate=['bro', 'number', 'person', 'number', 'qb', 'gone'], label=True),
 Phrase(original=['person', 'number', 'qb', 'go', 'draft'], candidate=['believe', 'person', 'went', 'number', 'qb', 'draft'], label=True),
 Phrase(original=['person', 'number', 'qb', 'go', 'draft'], candidate=['person', 'number', 'qb'], label=True)]

In [458]:
from gensim.corpora import Dictionary

def phrase_tokens(phrase):
    return [token for token in (phrase.original + phrase.candidate)]
    

vocab = Dictionary([phrase_tokens(p) for p in clean_train_data + clean_dev_data + clean_test_data])

len(vocab)

8787

In [459]:
VOCAB_SIZE = len(vocab) + 1 # +1 for padding

print(VOCAB_SIZE)

8788


In [460]:
id2token = dict([(i, token)for token, i in vocab.token2id.items()])

In [461]:
def text_to_sequence(words):
    return [i + 1 for i in vocab.doc2idx(words)]

def sequence_to_text(seq):
    return [id2token[i - 1] for i in seq if i]

In [462]:
def data_to_sequences(data):
    
    encoder_seqs = []
    decoder_seqs = []
    labels = []
    
    for phrase in data:
        encoder_seqs.append(text_to_sequence([t for t in phrase.original]))
        decoder_seqs.append(text_to_sequence([t for t in phrase.candidate]))
        labels.append(phrase.label)
        
    return encoder_seqs, decoder_seqs, labels 

train_encoder_seqs, train_decoder_seqs, train_labels = data_to_sequences(clean_train_data)

print(len(train_encoder_seqs))
print(len(train_decoder_seqs))
print(len(train_labels))

11530
11530
11530


In [463]:
print(sequence_to_text(train_encoder_seqs[0]))
print(sequence_to_text(train_decoder_seqs[0]))
print(train_labels[0])

['person', 'number', 'qb', 'go', 'draft']
['bro', 'number', 'person', 'number', 'qb', 'gone']
True


In [464]:
dev_encoder_seqs, dev_decoder_seqs, dev_labels = data_to_sequences(clean_dev_data)

In [465]:
test_encoder_seqs, test_decoder_seqs, test_labels = data_to_sequences(clean_test_data)

In [466]:
MAX_SEQ_LEN = max([len(seq) for seq in (train_encoder_seqs + train_decoder_seqs + \
                                       dev_encoder_seqs + dev_decoder_seqs + \
                                       test_decoder_seqs + test_encoder_seqs)])

print(MAX_SEQ_LEN)

13


In [467]:
from keras.preprocessing.sequence import pad_sequences

def padding(sequences):
    return pad_sequences(sequences, maxlen=MAX_SEQ_LEN, dtype='int32', padding='post', truncating='post')

In [493]:
EMBEDDING_SIZE = 500

In [494]:
import numpy as np

EMBEDDING_MATRIX = np.zeros((VOCAB_SIZE, EMBEDDING_SIZE))

def get_glove(word):
    try:
        return glove[word]
    except KeyError:
        pass
    return np.zeros(200)

def get_numberbatch(word):
    try:
        return numberbatch[word]
    except KeyError:
        pass
    return np.zeros(300)


missed = set([])

for word, i in vocab.token2id.items():   
    EMBEDDING_MATRIX[i] = np.append(get_numberbatch(word), get_glove(word))

print(len(missed))

0


In [452]:
print(missed)

{'toooo', 'ltrauzzi', 'itsstephsoricex', 'courside', 'fbis', '3pts', 'bmthofficial', 'prototypeversion', 'advisegiver', 'rulegoverence', '81st', 'jaibrooks1', 'spursvslakers', 'dayummmmmm', 'sticksaveswag', 'ccccrrrrraaaaazzzzzzyyyyyy', 'hmmmmmmmmmmm', '40000', 'latttwoone', 'babak1', 'guideshop', 'prayforkrista', 'izamakarewicz', 'daterape', 'cutiecalum', 'thooooo', 'meeee', 'haswellbased', 'dortmunddestruction', 'at127', 'squeeeee', 'mickid', 'madriddortmund', 'playpffs', 'concievec', 'mikereiss', 'burritolirry', '90s', 'favesssss', 'nouisniam', 'andforcing', 'hardinghe', '5th', '59th', 'ilyyyy', 'kinkybootsbway', '200th', '5050', '1future', 'thordarkworld', 'dunkfaced', 'airrielle', 'subtchicago', '3year', 'nofloorseats', 'gametheyre', 'lydiais', 'allllllllll', 'logic301', 'a1', '9inning', 'hannahbush', 'fivehole', 'saramcmann', '11th', 'pointwe', 'grooooooown', 'jonnysaraceno', '52nd', '27footers', '1', 'chuddyer', 'homewreaker', 'sdchargers', 'x10', 'latahhhh', 'nonhockey', 'sykes

In [470]:
len(missed)/len(vocab)

0.1139182883805622

In [471]:
train_data_missed = [(p, t) for p in clean_train_data for t in p.original + p.candidate if t in missed]

In [472]:
len(train_data_missed)/len(train_data)

0.3678230702515178

In [473]:
train_data_missed[:10]

[(Phrase(original=['person', 'number', 'qb', 'taken'], candidate=['bills', 'take', 'sportsteam', 'number', 'qb', 'board'], label=True),
  'sportsteam'),
 (Phrase(original=['person', 'number', 'qb', 'picked', 'overall'], candidate=['sportsteam', 'person', 'number', 'qb', 'taken'], label=True),
  'sportsteam'),
 (Phrase(original=['person', 'number', 'qb', 'picked', 'overall'], candidate=['boy', 'cbaire1', 'person', 'number', 'qb', 'taken'], label=True),
  'cbaire1'),
 (Phrase(original=['person', 'number', 'qb', 'taken', 'nfldraft'], candidate=['number', 'qb', 'taken', '17th', 'pick', 'geo', 'different'], label=True),
  '17th'),
 (Phrase(original=['person', 'number', 'qb', 'taken', 'nfldraft'], candidate=['person', 'go', 'NONE'], label=True),
  'NONE'),
 (Phrase(original=['person', 'number', 'qb', 'taken', 'nfldraft'], candidate=['sportsteam', 'person', 'number', 'qb', 'taken'], label=True),
  'sportsteam'),
 (Phrase(original=['person', 'number', 'qb', 'taken', 'huh'], candidate=['buffalo

In [495]:
from keras.layers import Embedding

embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                            output_dim = EMBEDDING_SIZE,
                            input_length = MAX_SEQ_LEN,
                            weights = [EMBEDDING_MATRIX], trainable = False)

In [496]:
from keras.layers import Input, LSTM, Embedding, TimeDistributed, Dense, Dropout
from keras.models import Model
from keras.optimizers import RMSprop

HIDDEN_DIM = 300

encoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
encoder_embedding = embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
decoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
decoder_embedding = embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
outputs = Dense(1, activation='sigmoid')(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)

model.summary()
model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 13, 500)      4394000     input_20[0][0]                   
                                                                 input_21[0][0]                   
__________________________________________________________________________________________________
lstm_33 (LSTM)                  [(None, 300), (None, 961200      embedding_7[0][0]                
__________

In [497]:
model.fit([padding(train_encoder_seqs), padding(train_decoder_seqs)], np.array(train_labels),
          batch_size = 100, epochs = 5, validation_data=([padding(dev_encoder_seqs), padding(dev_decoder_seqs)], dev_labels))

Train on 11530 samples, validate on 4142 samples
Epoch 1/5
11530/11530 [==============================] - 14s 1ms/step - loss: 0.6247 - acc: 0.6632 - val_loss: 0.6324 - val_acc: 0.6613
Epoch 2/5
11530/11530 [==============================] - 9s 743us/step - loss: 0.5882 - acc: 0.6970 - val_loss: 0.6305 - val_acc: 0.6649
Epoch 3/5
11530/11530 [==============================] - 9s 746us/step - loss: 0.5545 - acc: 0.7270 - val_loss: 0.6623 - val_acc: 0.6224
Epoch 4/5
11530/11530 [==============================] - 9s 749us/step - loss: 0.5405 - acc: 0.7383 - val_loss: 0.7017 - val_acc: 0.6564
Epoch 5/5
11530/11530 [==============================] - 9s 755us/step - loss: 0.5238 - acc: 0.7517 - val_loss: 0.6807 - val_acc: 0.6475


In [498]:
from sklearn.metrics import classification_report

print(classification_report(dev_labels, [prob.item() > 0.5 for prob in model.predict([padding(dev_encoder_seqs), padding(dev_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.67      0.88      0.76      2672
        True       0.51      0.22      0.31      1470

   micro avg       0.65      0.65      0.65      4142
   macro avg       0.59      0.55      0.54      4142
weighted avg       0.61      0.65      0.60      4142



In [503]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, [prob.item() > 0.5 for prob in model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.80      0.89      0.84       663
        True       0.28      0.16      0.20       175

   micro avg       0.74      0.74      0.74       838
   macro avg       0.54      0.53      0.52       838
weighted avg       0.69      0.74      0.71       838



In [500]:
def dump_scores(filename, predicted_similarity):
    with open(filename, 'w+') as f:
        for estimate in predicted_similarity:                    
            f.write("{}\t{:.4f}\n".format(str(estimate.item() > 0.5).lower(), estimate.item()))

In [501]:
dump_scores('PIT2015_zubovych_autoencoder.output', model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)]))

In [502]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py SemEval-PIT2015-py3/data/test_bin.label PIT2015_zubovych_autoencoder.output

838	zubovych	autoencoder		F: 0.204	Prec: 0.283	Rec: 0.160		P-corr: 0.111	F1: 0.348	Prec: 0.213	Rec: 0.943


In [511]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Activation, Dropout
from keras.layers import Embedding, Input
from keras.layers import LSTM, Lambda, concatenate
from keras import regularizers

HIDDEN_DIM=300

def exponent_neg_manhattan_distance(x, hidden_size=HIDDEN_DIM):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    return K.exp(-K.sum(K.abs(x[:,:hidden_size] - x[:,hidden_size:]), axis=1, keepdims=True))

def exponent_neg_cosine_distance(x, hidden_size=HIDDEN_DIM):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    leftNorm = K.l2_normalize(x[:,:hidden_size], axis=-1)
    rightNorm = K.l2_normalize(x[:,hidden_size:], axis=-1)
    return K.exp(K.sum(K.prod([leftNorm, rightNorm], axis=0), axis=1, keepdims=True))

In [529]:
seq_1 = Input(shape=(MAX_SEQ_LEN,), dtype='int32', name='sequence1')
seq_2 = Input(shape=(MAX_SEQ_LEN,), dtype='int32', name='sequence2')

input_1 = embedding_layer(seq_1)
input_2 = embedding_layer(seq_2)

l1 = LSTM(units=HIDDEN_DIM)

l1_out = l1(input_1)
l2_out = l1(input_2)

concats = concatenate([l1_out, l2_out], axis=-1)

#main_output = Lambda(exponent_neg_cosine_distance, output_shape=(1,))(concats)
#main_output = Lambda(exponent_neg_manhattan_distance, output_shape=(1,))(concats)
dense_ouput = Dense(2048, activation="relu")(concats)
main_output = Dense(1, activation="sigmoid")(dense_ouput)

model = Model(inputs=[seq_1, seq_2], outputs=[main_output])

opt = keras.optimizers.Adadelta(lr = 0.1, clipnorm=1.25)

#model.compile(optimizer=RMSprop(lr=1e-4), loss='mean_squared_error', metrics=['accuracy'])
#model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,loss='mean_squared_error', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence1 (InputLayer)          (None, 13)           0                                            
__________________________________________________________________________________________________
sequence2 (InputLayer)          (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 13, 500)      4394000     sequence1[0][0]                  
                                                                 sequence2[0][0]                  
__________________________________________________________________________________________________
lstm_40 (LSTM)                  (None, 300)          961200      embedding_7[12][0]               
          

In [ ]:
model.fit([padding(train_encoder_seqs), padding(train_decoder_seqs)], np.array(train_labels),
          batch_size = 100, epochs = 5, validation_data = ([padding(dev_encoder_seqs), padding(dev_decoder_seqs)], dev_labels))

Train on 11530 samples, validate on 4142 samples
Epoch 1/5


In [524]:
from sklearn.metrics import classification_report

print(classification_report(dev_labels, [prob.item() > 0.5 for prob in model.predict([padding(dev_encoder_seqs), padding(dev_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00      2672
        True       0.35      1.00      0.52      1470

   micro avg       0.35      0.35      0.35      4142
   macro avg       0.18      0.50      0.26      4142
weighted avg       0.13      0.35      0.19      4142



/home/szubovych/.virtualenvs/nlp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [525]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, [prob.item() > 0.5 for prob in model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       1.00      0.00      0.00       663
        True       0.21      1.00      0.35       175

   micro avg       0.21      0.21      0.21       838
   macro avg       0.60      0.50      0.17       838
weighted avg       0.83      0.21      0.07       838



In [526]:
dump_scores('PIT2015_zubovych_MaLSTM.output', model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)]))

In [527]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py SemEval-PIT2015-py3/data/test_bin.label PIT2015_zubovych_MaLSTM.output

838	zubovych	MaLSTM		F: 0.346	Prec: 0.209	Rec: 1.000		P-corr: 0.248	F1: 0.419	Prec: 0.322	Rec: 0.600
